# Model Training - FastText

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import os 
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity
import mlflow

import json
import requests
import boto3

import streamlit as st

import warnings
warnings.filterwarnings("ignore")

In [2]:
import yaml

def load_params():
    with open("../params.yaml") as f:
        params = yaml.safe_load(f)
    return params

params = load_params()

In [3]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

data = pd.read_pickle(params["model_pipeline"]["recipe_path"])
table = connect_database()

## Model Training

In [239]:
# fasttext_model = FastText(data.ingredients, **params["model_pipeline"]["fast_text"])
# fasttext_model.save(params["model_pipeline"]["model_path"])
fasttext_model = FastText.load(params["model_pipeline"]["model_path"])

In [5]:
# Quick sanity check
fasttext_model.wv.most_similar("peanut butter", topn=10)

[('peanut_butter', 0.8854959607124329),
 ('smooth_peanut_butter', 0.855765163898468),
 ('creamy_peanut_butter', 0.7880486249923706),
 ('salted_creamy_peanut_butter', 0.7332251667976379),
 ('creamy_salted_peanut_butter', 0.7154029607772827),
 ('nut_butter', 0.6878952383995056),
 ('salted_natural_peanut_butter', 0.6861165761947632),
 ('cocoa_butter', 0.6729941964149475),
 ('butter', 0.6723080277442932),
 ('natural_salted_peanut_butter', 0.653947651386261)]

### IDF (Inverse Document Frequency)

In [5]:
idf = {}
vector_embedding = {}
for i, vocab in enumerate(fasttext_model.wv.index_to_key):
    # Apply IDF
    n_recipe_contains_vocab = data.ingredients.apply(lambda x: vocab in x).sum()
    idf[vocab] = np.log(len(data.ingredients) / n_recipe_contains_vocab)

    # Apply IDF weight to Word2Vec embeddings
    vector_embedding[vocab] = fasttext_model.wv.vectors[i]/np.linalg.norm(fasttext_model.wv.vectors[i]) * idf[vocab] # Normalize w2v embedding first
    vector_embedding[vocab] = vector_embedding[vocab] / np.linalg.norm(vector_embedding[vocab])  # Normalize combined embedding again

idf

{'salt': 0.3928653931060427,
 'garlic': 0.9728863994977894,
 'water': 1.2474342884510103,
 'olive_oil': 1.3923734071962728,
 'black_pepper': 1.400105404479599,
 'sugar': 1.7065040189406901,
 'egg': 1.7908029890020982,
 'soy': 1.8923467061332238,
 'onion': 2.013707563137491,
 'ginger': 2.071498688165899,
 'vanilla_extract': 2.1601224114682016,
 'lemon': 2.1754537222149906,
 'unsalted_butter': 2.2039456780092967,
 'carrot': 2.3909022373074,
 'allpurpose_flour': 2.4993647329632305,
 'parsley': 2.522941463527227,
 'baking': 2.5289235352047745,
 'maple_syrup': 2.5349416075303375,
 'scallion': 2.571831036216051,
 'sesame_oil': 2.571831036216051,
 'milk': 2.6079667425452273,
 'pepper': 2.6298494537947352,
 'cinnamon': 2.717665659801652,
 'white_pepper': 2.7470795450079453,
 'cilantro': 2.759595352939776,
 'vegetable_oil': 2.7646458687258444,
 'oil': 2.7876942641621354,
 'cornstarch': 2.830028627988696,
 'lime': 2.846377765990226,
 'neutral_oil': 2.8518874218011954,
 'shaoxing_wine': 2.8999872

In [6]:
# Check if it's normalized
np.linalg.norm(vector_embedding.get("salt"))

1.0

### Model Scoring System

In [ ]:
# Prepare ingredient vectors
ingredient_vectors = []
for ingredients in data.ingredients:
    # Get the vectors for the recipe ingredients and normalize them
    embedding_vecs = [fasttext_model.wv.get_vector(ing, norm=True) for ing in ingredients]
    ingredient_vectors.append(embedding_vecs)

ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in ingredient_vectors])

In [242]:
# Prepare title vectors
title_vectors = []
for title in data.recipe_title:
    # Get the vector for the recipe title and normalize it
    title_split = title.split()
    embedding_vecs = [fasttext_model.wv.get_vector(title, norm=True) for title in title_split]
    title_vectors.append(embedding_vecs)

title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in title_vectors])

In [231]:
# Prediction scoring system
user_id = 110833230122006731136
n=10
text = "chicken thighs, rice, peanut butter, almond milk, kecap manis, soy sauce"
query = [ingredient.strip() for ingredient in text.split(",") if ingredient]
print("query:", query)

# Get the vectors for the search query and normalize them
query_vecs = np.array([fasttext_model.wv.get_vector(input, norm=True) for input in query])

# Calculate MaxSim for each recipe in the data
# Might be slow for large amount of data
ingredient_max_sim = np.array([])
for ing_vector in ingredient_vectors:
    tokens_sim = cosine_similarity(query_vecs, ing_vector)

    a_best = tokens_sim.max(axis=1)
    b_best = tokens_sim.max(axis=0)
    
    score = 0.5 * (a_best.mean() + b_best.mean())
    ingredient_max_sim = np.append(ingredient_max_sim, score)

# Calculate cosine similarity between the mean query vector and the mean ingredients vector
mean_query_vec = query_vecs.mean(axis=0)
ingredient_sim = cosine_similarity(mean_query_vec.reshape(1, -1), ingredient_vector_means)
title_sim = cosine_similarity(mean_query_vec.reshape(1, -1), title_vector_means)

score = (params["model_pipeline"]["model_scoring"]["w_cosine"] * ingredient_sim[0] + 
            params["model_pipeline"]["model_scoring"]["w_maxsim"] * ingredient_max_sim + 
            params["model_pipeline"]["model_scoring"]["w_title"] * title_sim[0])

rec_idx = np.argsort(score)[-params["model_service"]["n_recs"]:][::-1]
# rec_idx = [4557, 5297, 3977, 6109, 366, 3572, 6262, 314]
data.iloc[rec_idx]

query: ['chicken thighs', 'rice', 'peanut butter', 'almond milk', 'kecap manis', 'soy sauce']


,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
5297,5297,Rice Flour,https://thewoksoflife.com/how-to-make-rice-flour/,[rice],2.0,5.0,5.0,0.0,0.0,293.0,...,0.0,61.0,2.0,0.1,0.0,0.0,8.0,0.3,0.5,https://thewoksoflife.com/wp-content/uploads/2...
4557,4557,Rice S Outrageously Delicious Rice,https://www.recipetineats.com/rice-recipes-ric...,"[one_pot_greek_chicken, lemon_rice, mexican_be...",7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.recipetineats.com/tachyon/2020/03/...
5558,5558,Chicken Thighs,https://thewoksoflife.com/how-to-debone-chicke...,[chicken_thigh],4.0,10.0,10.0,0.0,0.0,211.0,...,74.0,198.0,0.0,0.0,75.0,0.0,8.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5484,5484,Down A Chicken,https://thewoksoflife.com/how-to-break-down-a-...,[chicken],9.0,10.0,10.0,0.0,0.0,409.0,...,133.0,360.0,0.0,0.0,267.0,3.0,21.0,2.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
4440,4440,12 Minute Thai Chicken Peanut Noodles,https://www.recipetineats.com/thai-chicken-pea...,"[oil, garlic, chicken_mince, curry, red_curry_...",10.0,12.0,3.0,9.0,0.0,581.0,...,950.0,1209.0,3.0,4.0,4585.0,11.0,84.0,6.0,0.0,https://www.recipetineats.com/tachyon/2021/01/...
4974,4974,Spicy Thai Chicken Stir Fry,https://www.recipetineats.com/spicy-thai-chick...,"[soy, dark_soy, fish, oyster, sugar, oil, thai...",6.0,15.0,7.0,8.0,0.0,427.0,...,791.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,230.0,https://www.recipetineats.com/tachyon/2016/11/...
4533,4533,Chinese Noodle Soup,https://www.recipetineats.com/chinese-noodle-s...,"[chicken_broth_low_sodium, garlic, ginger_piec...",6.0,15.0,5.0,10.0,0.0,352.0,...,1067.0,493.0,2.0,6.0,29.0,1.0,28.0,4.0,585.0,https://www.recipetineats.com/tachyon/2016/06/...
5191,5191,Sticky Chicken With Vegetable Fried Rice,https://www.recipetineats.com/sticky-chicken-v...,"[chicken_thigh, soy, tomato, rice_vinegar, hon...",15.0,20.0,10.0,10.0,0.0,421.0,...,947.0,295.0,3.9,11.0,3650.0,11.6,40.0,4.5,353.0,https://www.recipetineats.com/tachyon/2014/09/...
4635,4635,Thai Basil Chicken,https://www.recipetineats.com/thai-basil-chick...,"[chicken_thigh, green_onion, thai_basil_leaf, ...",6.0,15.0,10.0,5.0,0.0,360.0,...,588.0,231.0,1.0,2.0,148.0,2.0,14.0,1.0,160.0,https://www.recipetineats.com/tachyon/2017/03/...
6147,6147,Cantonese-Style Zongzi,https://thewoksoflife.com/zongzi-cantonese-style/,"[bamboo_leaf, short_grain_sticky_rice, soy, sa...",10.0,480.0,0.0,480.0,0.0,497.0,...,607.0,221.0,2.0,1.0,120.0,0.2,26.0,1.9,0.0,https://thewoksoflife.com/wp-content/uploads/2...


### DynamoDB Data Revtrieval

In [10]:
# user_id = 110833230122006731136
# table_test = table.get_item(Key={"user_id": int(user_id)})["Item"]

# table_test

In [11]:
# from datetime import datetime
# test_map = {"liked_idx": list(map(int, list(liked_idx))), 
#             "disliked_idx": list(map(int, list(disliked_idx))),
#             "date": datetime.now().strftime("%Y-%m-%d" + "%H:%M:%S")}
# test_map


## Wrap Model with MLFlow Python Model

Models from Code (Add set model) or Load as Class (Restart kernel after every overwrite)

In [243]:
class RecipeFastText(mlflow.pyfunc.PythonModel):
    def __init__(self, params):
        self.params = params

    def load_context(self, context):
        self.model = FastText.load(context.artifacts["model_path"])
        self.data = pd.read_pickle(context.artifacts["data_path"])

        # Prepare ingredient vectors
        self.ingredient_vectors = []
        for ingredients in self.data.ingredients:
            # Get the vectors for the recipe ingredients and normalize them
            embedding_vecs = [self.model.wv.get_vector(ing, norm=True) for ing in ingredients]
            self.ingredient_vectors.append(embedding_vecs)

        self.ingredient_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in self.ingredient_vectors])

        # Prepare title vectors
        self.title_vectors = []
        for title in self.data.recipe_title:
            # Get the vector for the recipe title and normalize it
            title_split = title.split()
            embedding_vecs = [self.model.wv.get_vector(title, norm=True) for title in title_split]
            self.title_vectors.append(embedding_vecs)

        self.title_vector_means = np.array([np.mean(ing_vector, axis=0) for ing_vector in self.title_vectors])

    def predict(self, model_input: list[str]) -> list[float]:
        """Predicts the recipe similarity score based on the input string."""
        
        # Get the vectors for the search query and normalize them
        query_vecs = np.array([self.model.wv.get_vector(input, norm=True) for input in query])

        # Calculate MaxSim for each recipe in the data
        # Might be slow for large amount of data
        ingredient_max_sim = np.array([])
        for ing_vector in self.ingredient_vectors:
            tokens_sim = cosine_similarity(query_vecs, ing_vector)

            a_best = tokens_sim.max(axis=1)
            b_best = tokens_sim.max(axis=0)
            
            score = 0.5 * (a_best.mean() + b_best.mean())
            ingredient_max_sim = np.append(ingredient_max_sim, score)

        # Calculate cosine similarity between the mean query vector and the mean ingredients vector
        mean_query_vec = query_vecs.mean(axis=0)
        ingredient_sim = cosine_similarity(mean_query_vec.reshape(1, -1), ingredient_vector_means)
        title_sim = cosine_similarity(mean_query_vec.reshape(1, -1), title_vector_means)
        
        score = (self.params["model_scoring"]["w_cosine"] * ingredient_sim[0] + 
                self.params["model_scoring"]["w_maxsim"] * ingredient_max_sim + 
                self.params["model_scoring"]["w_title"] * title_sim[0])

        return score

### Log Model to MLFlow

In [244]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("FastText-Model")

<Experiment: artifact_location='mlflow-artifacts:/198067954999234058', creation_time=1756094574349, experiment_id='198067954999234058', last_update_time=1756094574349, lifecycle_stage='active', name='FastText-Model', tags={}>

In [249]:
# Load params
params = load_params()

with mlflow.start_run():
    # Load data
    data = pd.read_pickle(params["model_pipeline"]["recipe_path"])

    # Load fasttext model
    fasttext_model = RecipeFastText(params["model_pipeline"])

    # Log parameters
    mlflow.log_params(params["model_pipeline"])

    # Log model
    model_info = mlflow.pyfunc.log_model(
        name="fasttext_model",
        python_model=fasttext_model,
        artifacts={"model_path": params["model_pipeline"]["model_path"],
                   "model_ngram_path": params["model_pipeline"]["model_ngram_path"],
                   "data_path": params["model_pipeline"]["recipe_path"]},
        pip_requirements=["gensim==4.3.3"]
    )

🏃 View run bright-calf-932 at: http://127.0.0.1:8080/#/experiments/198067954999234058/runs/d41dd113c6114f21a3ae2e9da0ea1b03
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/198067954999234058


In [250]:
local_uri = os.path.join("../mlflow/mlartifacts", *model_info.artifact_path.split("/")[1:])

metadata = {
    "server_uri": model_info.model_uri,
    "local_uri": local_uri,
    "run_id": model_info.run_id,
    "time_logged": model_info.utc_time_created,
    "model_params": params["model_pipeline"]
}

with open("../experiment_metadata.json", "w") as f:
    f.write(json.dumps(metadata, indent=4))

In [ ]:
with open("../experiment_metadata.json", "r") as f:
    metadata = json.load(f)

model = mlflow.pyfunc.load_model(metadata["local_uri"])

In [248]:
rec_idx = model.predict(["peanut butter", "jelly", "bread"])  # Example usage
rec_idx = np.argsort(rec_idx)[::-1] # Get top recommendations
data.iloc[rec_idx]

,id,recipe_title,recipe_url,ingredients,num_steps,total_time,prep_time,cook_time,custom_time,calories,...,sodium,potassium,fiber,sugar,vitamin_a,vitamin_c,calcium,iron,serving_size,image_url
5297,5297,Rice Flour,https://thewoksoflife.com/how-to-make-rice-flour/,[rice],2.0,5.0,5.0,0.0,0.0,293.0,...,0.0,61.0,2.0,0.1,0.0,0.0,8.0,0.3,0.5,https://thewoksoflife.com/wp-content/uploads/2...
4557,4557,Rice S Outrageously Delicious Rice,https://www.recipetineats.com/rice-recipes-ric...,"[one_pot_greek_chicken, lemon_rice, mexican_be...",7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.recipetineats.com/tachyon/2020/03/...
5558,5558,Chicken Thighs,https://thewoksoflife.com/how-to-debone-chicke...,[chicken_thigh],4.0,10.0,10.0,0.0,0.0,211.0,...,74.0,198.0,0.0,0.0,75.0,0.0,8.0,1.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
5484,5484,Down A Chicken,https://thewoksoflife.com/how-to-break-down-a-...,[chicken],9.0,10.0,10.0,0.0,0.0,409.0,...,133.0,360.0,0.0,0.0,267.0,3.0,21.0,2.0,0.0,https://thewoksoflife.com/wp-content/uploads/2...
4440,4440,12 Minute Thai Chicken Peanut Noodles,https://www.recipetineats.com/thai-chicken-pea...,"[oil, garlic, chicken_mince, curry, red_curry_...",10.0,12.0,3.0,9.0,0.0,581.0,...,950.0,1209.0,3.0,4.0,4585.0,11.0,84.0,6.0,0.0,https://www.recipetineats.com/tachyon/2021/01/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248,1248,Tortellini Salad,https://www.loveandlemons.com/tortellini-salad/,"[cheese_tortellini, italian_dressing, cherry_t...",3.0,25.0,20.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...
1444,1444,Cucumber Tomato Salad,https://www.loveandlemons.com/cucumber-tomato-...,"[persian_cucumber, cherry_tomato, halloumi_che...",3.0,30.0,15.0,15.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...
4034,4034,Pizza Dough,https://natashaskitchen.com/how-to-freeze-pizz...,"[pizza_dough_recipe, olive_oil]",7.0,5.0,5.0,0.0,0.0,258.0,...,707.0,1.0,1.0,6.0,0.0,0.0,1.0,3.0,0.0,https://natashaskitchen.com/wp-content/uploads...
1222,1222,Grilled Halloumi Cheese,https://www.loveandlemons.com/grilled-halloumi...,"[halloumi_cheese, olive_oil]",1.0,11.0,5.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://cdn.loveandlemons.com/wp-content/uploa...


## Model Register

In [22]:
with mlflow.start_run():
    mlflow.register_model(
        model_uri=model_info._model_uri,
        name="fasttext_model"
    )

Successfully registered model 'fasttext_model'.
2025/08/23 17:04:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: fasttext_model, version 1


🏃 View run rumbling-snail-782 at: http://127.0.0.1:8080/#/experiments/329658568731902423/runs/769ba51e2ca54d7080dd69d61e90b4bd
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/329658568731902423


Created version '1' of model 'fasttext_model'.


In [23]:
model = mlflow.pyfunc.load_model("models:/fasttext_model/latest")